In [ ]:
#!pip install hawkesbook
#!pip install newsapi-python
#!pip install pytrends

In [ ]:
# -*- coding: utf-8 -*-
import requests # Utilisé pour les requêtes manuelles à Polygon
import pandas as pd
import numpy as np
import datetime as dt
import time
from pytrends.request import TrendReq
import pandas_datareader.data as web
# NLTK n'est plus nécessaire car nous n'utilisons plus NewsAPI pour le sentiment
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import itertools
import os
import warnings
import hawkesbook as hl

In [ ]:
# --- Configuration ---
# ATTENTION: Remplacez par votre vraie clé API !
POLYGON_API_KEY = "YOUR API KEY" # Mettez votre clé ici
# NEWSAPI_KEY n'est plus nécessaire

# Actions NASDAQ (Exemple)
TICKERS = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'TSLA', 'MRNA', 'SBUX', 'CMCSA']
N_STOCKS = len(TICKERS)

# Période d'analyse
END_DATE = dt.date.today()
START_DATE = END_DATE - dt.timedelta(days=365*2) # 1 an

# Paramètres temporels
TIME_STEP = 5 # minutes
ROLLING_VOL_WINDOW_DAYS = 7
EPISODE_DURATION_WEEKS = 1
PAGINATION_RANGE_DAYS = 30 # Récupérer les données Polygon par tranches de 30 jours pour éviter les limites

# Paramètres de détection d'événements
VOLATILITY_THRESHOLD_QUANTILE = 0.92

# Paramètres Hawkes (pour l'estimation MLE)
INITIAL_LAMBDA = np.ones(N_STOCKS) * 0.1
INITIAL_ALPHA = np.ones((N_STOCKS, N_STOCKS)) * 0.05
np.fill_diagonal(INITIAL_ALPHA, 0.1)
INITIAL_BETA = np.ones(N_STOCKS) * 1.0
INITIAL_THETA = (INITIAL_LAMBDA, INITIAL_ALPHA, INITIAL_BETA)

# Indicateurs Macroéconomiques (FRED) - Ajout de VXN
FRED_INDICATORS = {
    'VIX': 'VIXCLS',       # Indice de Volatilité CBOE
    'VXN': 'VXNCLS',       # Indice de Volatilité NASDAQ-100
    'UNRATE': 'UNRATE',    # Taux de chômage US
    'CPI': 'CPIAUCSL',     # Indice des prix à la consommation US
    'FEDFUNDS': 'FEDFUNDS' # Taux des fonds fédéraux
}

# Keywords pour Google Trends - Ajout de comparaisons par paire
TRENDS_KEYWORDS_PER_STOCK = {ticker: [ticker] for ticker in TICKERS}
TRENDS_KEYWORDS_THEMES = [
    'semiconductor shortage', 'AI regulation', 'cloud computing competition', 'supply chain disruption',
    'Apple vs Microsoft', 'Nvidia vs AMD', 'Tesla vs Ford' # Exemples de paires
]

In [ ]:
# --- Fonctions ---

def fetch_polygon_data_manual(tickers, start_date, end_date, api_key, timespan='minute', multiplier=5, pagination_days=30):
    """Récupère les données de prix historiques de Polygon.io manuellement avec requests, gère la pagination et les limites de taux."""
    print(f"Récupération manuelle des données Polygon pour {len(tickers)} tickers...")
    base_url = "https://api.polygon.io"
    all_ticker_data = {ticker: [] for ticker in tickers}
    requests_count = 0
    minute_start_time = time.time()

    # Headers pour l'authentification
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    # Boucle sur les tickers
    for ticker in tqdm(tickers, desc="Tickers"):
        current_start_date = start_date
        # Boucle de pagination par date
        while current_start_date <= end_date:
            # Définir la fin de la période de pagination
            current_end_date = min(current_start_date + dt.timedelta(days=pagination_days - 1), end_date)
            start_str = current_start_date.strftime("%Y-%m-%d")
            end_str = current_end_date.strftime("%Y-%m-%d")

            # Construire l'URL de l'endpoint
            endpoint = f"/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_str}/{end_str}"
            url = base_url + endpoint
            params = {"adjusted": "true", "sort": "asc", "limit": 50000} # Max limit

            # --- Gestion de la limite de taux (5 requêtes/minute pour le plan gratuit) ---
            current_time = time.time()
            if requests_count >= 4 and (current_time - minute_start_time) < 60:
                sleep_time = 60 - (current_time - minute_start_time) + 1 # +1 pour la marge
                print(f"Limite de taux atteinte, attente de {sleep_time:.1f} secondes...")
                time.sleep(sleep_time)
                requests_count = 0
                minute_start_time = time.time()
            elif (current_time - minute_start_time) >= 60:
                 requests_count = 0
                 minute_start_time = time.time()
            # --------------------------------------------------------------------------

            try:
                response = requests.get(url, headers=headers, params=params)
                requests_count += 1
                response.raise_for_status()
                data = response.json()

                if data.get("resultsCount", 0) > 0 and 'results' in data:
                    df = pd.DataFrame(data['results'])
                    df = df.rename(columns={'t': 'timestamp', 'c': ticker})
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
                    df = df.set_index('timestamp')
                    all_ticker_data[ticker].append(df[[ticker]])
                # Même si les résultats sont vides, on avance la date
                current_start_date = current_end_date + dt.timedelta(days=1)
                # Petite pause entre chaque requête individuelle pour être sûr
                time.sleep(0.5)

            except requests.exceptions.RequestException as e:
                print(f"Erreur lors de la requête pour {ticker} ({start_str} à {end_str}): {e}")
                if response is not None:
                     print(f"Status Code: {response.status_code}, Response: {response.text[:200]}") # Affiche début de la réponse si erreur
                     if response.status_code == 429: # Rate limit explicite
                         print("Rate limit (429) détecté, attente de 65 secondes...")
                         time.sleep(65)
                         requests_count = 0 # Réinitialiser le compteur après attente
                         minute_start_time = time.time()
                         continue # Retenter la même requête
                # Si autre erreur, on saute cette période pour ce ticker mais on continue
                current_start_date = current_end_date + dt.timedelta(days=1)
                time.sleep(2) # Pause après une erreur

            except Exception as e_json:
                 print(f"Erreur de décodage JSON ou autre pour {ticker} ({start_str} à {end_str}): {e_json}")
                 # Si erreur, on saute cette période pour ce ticker mais on continue
                 current_start_date = current_end_date + dt.timedelta(days=1)
                 time.sleep(2)


    # Combiner les données pour chaque ticker puis tous les tickers
    final_dfs = []
    for ticker in tickers:
        if all_ticker_data[ticker]:
            ticker_df = pd.concat(all_ticker_data[ticker]).sort_index()
            # Supprimer les duplicatas d'index si la pagination a des chevauchements minimes
            ticker_df = ticker_df[~ticker_df.index.duplicated(keep='first')]
            final_dfs.append(ticker_df)
        else:
            print(f"Aucune donnée valide récupérée pour {ticker}")

    if not final_dfs:
        print("Aucune donnée n'a pu être récupérée pour aucun ticker.")
        return pd.DataFrame()

    # Combiner tous les tickers
    full_df = pd.concat(final_dfs, axis=1)
    # Remplir les NaNs (peut arriver si les tickers n'ont pas exactement les mêmes timestamps)
    full_df = full_df.ffill().bfill()
    print("Données Polygon récupérées manuellement.")
    return full_df


def calculate_volatility_and_events(price_df, window_days, timestep_min, quantile_threshold):
    """Calcule la volatilité et détecte les événements de forte volatilité."""
    print("Calcul de la volatilité et détection des événements...")
    log_returns = np.log(price_df / price_df.shift(1)).dropna()

    trading_periods_per_day = (6.5 * 60) / timestep_min # Approximation heures de trading (ex: 9:30-16:00 ET)
    window_size = int(window_days * trading_periods_per_day)

    if window_size <= 1:
        raise ValueError("Fenêtre de volatilité trop petite.")

    rolling_vol = log_returns.rolling(window=window_size, min_periods=int(window_size*0.8)).std().dropna() # min_periods ajouté

    events = []
    thresholds = {}
    for ticker in price_df.columns:
        if ticker not in rolling_vol.columns or rolling_vol[ticker].isnull().all():
            print(f"Pas de données de volatilité pour {ticker}, il est ignoré.")
            continue
        vol = rolling_vol[ticker]
        try:
            threshold = vol.quantile(quantile_threshold)
            thresholds[ticker] = threshold
        except Exception as e:
             print(f"Impossible de calculer le quantile pour {ticker}, seuil mis à NaN: {e}")
             thresholds[ticker] = np.nan
             continue # Ne pas chercher d'événements si seuil non calculable

        if pd.isna(threshold): continue

        high_vol_times = vol[vol > threshold].index
        ticker_id = list(price_df.columns).index(ticker)
        for t in high_vol_times:
            time_in_days = (t - price_df.index[0]).total_seconds() / (24 * 60 * 60)
            events.append({'timestamp': t, 'time_days': time_in_days, 'ticker': ticker, 'ticker_id': ticker_id})

    print(f"Détection de {len(events)} événements de forte volatilité.")
    if not events:
        print("Aucun événement détecté.")
        return pd.DataFrame(), pd.DataFrame(), {}

    events_df = pd.DataFrame(events).sort_values('timestamp').reset_index(drop=True)
    return rolling_vol, events_df, thresholds

# Fonction pour récupérer les tendances Google - inchangée mais utilisera les keywords étendus
def fetch_trends_data(keywords_dict, themes, start_date, end_date, episode_starts):
    """Récupère les données Google Trends agrégées par épisode."""
    print("Récupération des données Google Trends...")
    pytrends = TrendReq(hl='en-US', tz=360)
    trends_data = []

    all_keywords = list(itertools.chain(*keywords_dict.values())) + themes
    # Attention : certains keywords peuvent ne retourner aucune donnée.
    # Google Trends limite à 5 keywords par requête
    kw_groups = [all_keywords[i:i + 5] for i in range(0, len(all_keywords), 5)]

    trends_df = pd.DataFrame()
    timeframe = f"{start_date.strftime('%Y-%m-%d')} {end_date.strftime('%Y-%m-%d')}"

    for kw_group in tqdm(kw_groups, desc="Fetching Trends"):
        # Filtrer les éventuels keywords vides ou None
        kw_group = [kw for kw in kw_group if kw]
        if not kw_group: continue
        try:
            pytrends.build_payload(kw_group, cat=0, timeframe=timeframe, geo='', gprop='')
            interest_over_time_df = pytrends.interest_over_time()
            if 'isPartial' in interest_over_time_df.columns:
                 interest_over_time_df = interest_over_time_df.drop(columns=['isPartial'])

            if trends_df.empty:
                trends_df = interest_over_time_df
            else:
                # Join en gérant les colonnes manquantes dans l'un ou l'autre
                trends_df = trends_df.join(interest_over_time_df, how='outer')
            time.sleep(2)
        except Exception as e:
            print(f"Erreur Pytrends pour {kw_group}: {e}.")
            # Ajouter des colonnes vides si nécessaire pour la cohérence
            for kw in kw_group:
                if kw not in trends_df.columns: trends_df[kw] = np.nan

    if trends_df.empty:
        print("Aucune donnée Google Trends trouvée.")
        return pd.DataFrame()

    # Agréger par épisode (moyenne)
    # Note : resample fonctionne mieux avec un index DatetimeIndex
    trends_df.index = pd.to_datetime(trends_df.index)
    trends_agg = trends_df.resample(f'{EPISODE_DURATION_WEEKS*7}D').mean()

    # Aligner sur les débuts d'épisodes et remplir
    trends_agg = trends_agg.reindex(episode_starts, method='ffill').fillna(0)
    trends_agg.columns = ['Trend_'+col.replace(' ', '_') for col in trends_agg.columns] # Remplacer espaces

    print("Données Google Trends agrégées.")
    return trends_agg

# Fonction pour récupérer les données macro FRED - inchangée mais utilisera les indicateurs étendus
def fetch_macro_data(fred_indicators, start_date, end_date, episode_starts):
    """Récupère les données macroéconomiques de FRED agrégées par épisode."""
    print("Récupération des données macroéconomiques FRED...")
    try:
        # Essayer de récupérer les données
        macro_df = web.DataReader(list(fred_indicators.values()), 'fred', start_date, end_date)
        # Renommer les colonnes avec les clés fournies
        name_map = {v: k for k, v in fred_indicators.items()}
        macro_df = macro_df.rename(columns=name_map)

        # Agréger par épisode (prendre la dernière valeur)
        # Assurer que l'index est un DatetimeIndex
        macro_df.index = pd.to_datetime(macro_df.index)
        macro_agg = macro_df.resample(f'{EPISODE_DURATION_WEEKS*7}D').last()

        # Aligner sur les débuts d'épisodes et remplir les NaNs
        macro_agg = macro_agg.reindex(episode_starts, method='ffill')
        macro_agg = macro_agg.fillna(method='ffill').fillna(method='bfill') # Fill forward puis backward

        macro_agg.columns = ['Macro_'+col for col in macro_agg.columns]
        print("Données macroéconomiques agrégées.")
        return macro_agg
    except Exception as e:
        print(f"Erreur lors de la récupération des données FRED: {e}")
        # Retourner un DataFrame vide avec les colonnes attendues si erreur
        cols = ['Macro_'+k for k in fred_indicators.keys()]
        return pd.DataFrame(columns=cols, index=episode_starts)

# Nouvelle fonction pour calculer les corrélations glissantes par épisode
def calculate_rolling_correlations(price_df, episode_starts):
    """Calcule la corrélation des log-rendements entre les paires d'actions pour chaque épisode."""
    print("Calcul des corrélations par épisode...")
    log_returns = np.log(price_df / price_df.shift(1)).dropna()
    correlation_features = []

    ticker_pairs = list(itertools.combinations(price_df.columns, 2))
    column_names = [f"Corr_{p[0]}_{p[1]}" for p in ticker_pairs]

    for i in range(len(episode_starts)):
        ep_start = episode_starts[i]
        ep_end = episode_starts[i+1] if i+1 < len(episode_starts) else log_returns.index.max() + dt.timedelta(microseconds=1)

        episode_returns = log_returns[(log_returns.index >= ep_start) & (log_returns.index < ep_end)]

        correlations = {}
        if len(episode_returns) > 1: # Besoin d'au moins 2 points pour la corrélation
            corr_matrix = episode_returns.corr()
            for pair in ticker_pairs:
                col_name = f"Corr_{pair[0]}_{pair[1]}"
                # Vérifier si les tickers existent dans la matrice (peuvent être absents si pas de données)
                if pair[0] in corr_matrix.columns and pair[1] in corr_matrix.columns:
                     correlations[col_name] = corr_matrix.loc[pair[0], pair[1]]
                else:
                     correlations[col_name] = np.nan # Mettre NaN si un ticker manque
        else:
             for pair in ticker_pairs:
                col_name = f"Corr_{pair[0]}_{pair[1]}"
                correlations[col_name] = np.nan # Mettre NaN si pas assez de données

        correlations['episode_start'] = ep_start
        correlation_features.append(correlations)

    if not correlation_features:
        return pd.DataFrame(columns=['episode_start'] + column_names).set_index('episode_start')

    corr_df = pd.DataFrame(correlation_features).set_index('episode_start')
    # Remplir les NaNs (par ex., si épisode avec < 2 retours) avec 0 ou moyenne ? Prenons 0.
    corr_df = corr_df.fillna(0)
    print("Corrélations par épisode calculées.")
    return corr_df


# Fonction d'estimation Hawkes - inchangée
def estimate_hawkes_for_episode(events_in_episode, n_stocks, T_episode_days, initial_theta):
    """Estime les paramètres Hawkes pour un épisode donné."""
    if events_in_episode.empty or len(events_in_episode) < 2:
        nan_params = np.full(n_stocks + n_stocks**2 + n_stocks, np.nan)
        return nan_params

    times = events_in_episode['time_days'].values
    ids = events_in_episode['ticker_id'].values
    times = times - times[0]
    T = max(T_episode_days, times[-1] + 1e-6) # Assurer T >= max(time)

    try:
        # print(f"Estimating MLE for episode (T={T:.2f} days, N={len(times)} events)...")
        estimated_theta, log_likelihood = hl.mutual_exp_mle(times, ids, T, initial_theta)
        # print(f"Log-Likelihood: {log_likelihood:.2f}")
        flat_theta = hl.flatten_theta(estimated_theta)
        return flat_theta
    except Exception as e:
        print(f"Erreur durant l'estimation MLE: {e}")
        nan_params = np.full(n_stocks + n_stocks**2 + n_stocks, np.nan)
        return nan_params

# Fonction de plot - inchangée
def plot_point_process(events_df, tickers):
    """Affiche le point process multivarié."""
    plt.figure(figsize=(15, 6))
    # Utiliser les timestamps directement pour l'axe des x
    plt.scatter(events_df['timestamp'], events_df['ticker_id'], alpha=0.6, s=10)
    plt.yticks(range(len(tickers)), tickers)
    plt.xlabel("Date")
    plt.ylabel("Action")
    plt.title("Processus Ponctuel des Événements de Forte Volatilité (7 jours)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Récupérer les données de prix (version manuelle)
price_data = fetch_polygon_data_manual(TICKERS, START_DATE, END_DATE, POLYGON_API_KEY,
                                     multiplier=TIME_STEP, pagination_days=PAGINATION_RANGE_DAYS)

if price_data.empty or price_data.shape[1] < N_STOCKS:
    print(f"Données de prix incomplètes ou manquantes (obtenues: {price_data.shape[1]}/{N_STOCKS}). Arrêt.")
    exit()

# S'assurer que l'index est trié
price_data = price_data.sort_index()

In [ ]:
print(type(price_data))
print(price_data.shape)
print(365 * 10 * 12)

In [ ]:
# 2. Calculer la volatilité et détecter les événements
rolling_vol, events_df, vol_thresholds = calculate_volatility_and_events(
    price_data,
    ROLLING_VOL_WINDOW_DAYS,
    TIME_STEP,
    VOLATILITY_THRESHOLD_QUANTILE
)

if events_df.empty:
    print("Impossible de continuer sans événements de volatilité.")
    exit()

# Afficher le processus ponctuel
plot_point_process(events_df, list(price_data.columns)) # Utiliser les colonnes réelles de price_data

In [ ]:
# 3. Définir les épisodes de 2 semaines
episode_duration_td = dt.timedelta(weeks=EPISODE_DURATION_WEEKS)
# S'assurer que les épisodes commencent bien après le début des données pour la vol glissante
start_offset = dt.timedelta(days=ROLLING_VOL_WINDOW_DAYS + 1) # Ajouter une marge
analysis_start_date = price_data.index.min() + start_offset
episode_starts = pd.date_range(start=analysis_start_date.normalize(),
                               end=price_data.index.max().normalize(),
                               freq=episode_duration_td)

if episode_starts.empty:
     print("Aucun épisode complet trouvé après la période initiale de calcul de volatilité. Vérifiez les dates.")
     exit()

print(f"Nombre d'épisodes de {EPISODE_DURATION_WEEKS} semaines: {len(episode_starts)}")

In [ ]:
# 4. Récupérer et agréger les indicateurs alternatifs
trends_agg = fetch_trends_data(TRENDS_KEYWORDS_PER_STOCK, TRENDS_KEYWORDS_THEMES, START_DATE, END_DATE, episode_starts)
macro_agg = fetch_macro_data(FRED_INDICATORS, START_DATE, END_DATE, episode_starts)
corr_agg = calculate_rolling_correlations(price_data, episode_starts)

# Fusionner tous les indicateurs agrégés
# news_agg est retiré
all_indicators = pd.concat([trends_agg, macro_agg, corr_agg], axis=1)

# Remplir les NaNs restants (peut arriver si une API a eu des erreurs)
all_indicators = all_indicators.fillna(method='ffill').fillna(method='bfill').fillna(0)

print("Indicateurs agrégés:")
#print(all_indicators.head())
print(all_indicators.shape)
print("Colonnes d'indicateurs:", all_indicators.columns)


# --- Note sur GDELT ---
#print("\nNote: Pour une analyse d'actualités open-source plus poussée (co-mentions, sentiment),")
#print("considérez GDELT Project. Son intégration demande un travail de traitement de données conséquent.")
# ---------------------

In [ ]:
len(episode_starts)

In [ ]:
# 5. Estimer les paramètres Hawkes pour chaque épisode
estimated_params_list = []
episode_data_for_regression = []

# Utiliser une copie des colonnes réelles pour l'ID ticker
actual_tickers = list(price_data.columns)
ticker_to_id = {ticker: i for i, ticker in enumerate(actual_tickers)}
events_df['ticker_id'] = events_df['ticker'].map(ticker_to_id) # Recalculer ID basé sur les tickers présents

# Ajuster N_STOCKS si certains tickers manquaient
N_STOCKS_ACTUAL = len(actual_tickers)
if N_STOCKS_ACTUAL != N_STOCKS:
    print(f"Ajustement du nombre d'actions à {N_STOCKS_ACTUAL} en raison de données manquantes.")
    # Recréer les paramètres initiaux avec la bonne dimension si nécessaire
    INITIAL_LAMBDA = np.ones(N_STOCKS_ACTUAL) * 0.1
    INITIAL_ALPHA = np.ones((N_STOCKS_ACTUAL, N_STOCKS_ACTUAL)) * 0.05
    np.fill_diagonal(INITIAL_ALPHA, 0.1)
    INITIAL_BETA = np.ones(N_STOCKS_ACTUAL) * 1.0
    INITIAL_THETA = (INITIAL_LAMBDA, INITIAL_ALPHA, INITIAL_BETA)


for i in tqdm(range(len(episode_starts)), desc="Estimating Hawkes per Episode"):
    ep_start_ts = episode_starts[i]
    ep_end_ts = episode_starts[i+1] if i+1 < len(episode_starts) else price_data.index.max() + dt.timedelta(microseconds=1)

    mask = (events_df['timestamp'] >= ep_start_ts) & (events_df['timestamp'] < ep_end_ts)
    events_in_episode = events_df[mask].copy() # Utiliser une copie pour éviter SettingWithCopyWarning

    if events_in_episode.empty:
        # print(f"Pas d'événements pour l'épisode commençant le {ep_start_ts}, ignoré.")
        continue

    T_episode = (ep_end_ts - ep_start_ts).total_seconds() / (24 * 60 * 60)

    flat_params = estimate_hawkes_for_episode(events_in_episode, N_STOCKS_ACTUAL, T_episode, INITIAL_THETA)

    if not np.isnan(flat_params).any() and ep_start_ts in all_indicators.index:
         episode_data_for_regression.append({
             'episode_start': ep_start_ts,
             'hawkes_params': flat_params,
             'indicators': all_indicators.loc[ep_start_ts].values
         })
    # else:
        # Gérer les cas d'échec ou d'indicateurs manquants si nécessaire (déjà loggé dans estimate_hawkes...)

In [ ]:
print(type(episode_data_for_regression))
print(len(episode_data_for_regression)) # 17 -> 17 épisodes de 2 semaines durant lesquels le processus contient des evenements
print(type(episode_data_for_regression[0]))
print(episode_data_for_regression[0].keys())
print(type(episode_data_for_regression[0]['episode_start']))
print(type(episode_data_for_regression[0]['hawkes_params']))
print(type(episode_data_for_regression[0]['indicators']))

H1 = episode_data_for_regression[0]
print(len(H1))

print(H1['indicators'].shape)
print(H1['hawkes_params'].shape)
np.set_printoptions(precision=5, suppress=True)
pd.set_option('display.float_format', '{:.5f}'.format)

print(H1['hawkes_params'])

In [ ]:
# 6. Préparer les données pour la régression
if not episode_data_for_regression:
    print("Aucune donnée valide pour entraîner le modèle de régression.")
    exit()

X_list = [item['indicators'] for item in episode_data_for_regression]
Y_list = [item['hawkes_params'] for item in episode_data_for_regression]

# S'assurer que toutes les lignes d'indicateurs ont la même forme
expected_len = len(all_indicators.columns)
X_list_filtered = [x for x in X_list if len(x) == expected_len]
Y_list_filtered = [y for x, y in zip(X_list, Y_list) if len(x) == expected_len]

if len(X_list_filtered) != len(X_list):
     print(f"Avertissement: {len(X_list) - len(X_list_filtered)} épisodes exclus en raison d'un nombre incohérent d'indicateurs.")

if not X_list_filtered:
     print("Aucun indicateur cohérent trouvé pour la régression.")
     exit()

X = pd.DataFrame(X_list_filtered, columns=all_indicators.columns)
# Créer des noms pour les paramètres Hawkes (potentiellement ajustés par N_STOCKS_ACTUAL)
param_names = [f'lambda_{i}' for i in range(N_STOCKS_ACTUAL)] + \
              [f'alpha_{i}_{j}' for i in range(N_STOCKS_ACTUAL) for j in range(N_STOCKS_ACTUAL)] + \
              [f'beta_{i}' for i in range(N_STOCKS_ACTUAL)]
Y = pd.DataFrame(Y_list_filtered, columns=param_names)

print("\nPréparation des données pour la régression:")
print("X (indicateurs) shape:", X.shape)
print("Y (paramètres Hawkes) shape:", Y.shape)
# print("X head:\n", X.head())
# print("Y head:\n", Y.head())

In [ ]:
# --- Exécution Principale ---


# 7. Entraîner le modèle de régression et valider

# Standardiser les features X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
X_scaled_const = sm.add_constant(X_scaled, has_constant='add') # Ajoute une colonne de 1s

regression_results = {}

# Exemple : Prédire le premier terme d'interaction alpha_0_1
target_param = f'alpha_{0}_{1}' if N_STOCKS_ACTUAL > 1 else f'alpha_{0}_{0}' # Assurer que l'index existe

if target_param not in Y.columns:
    print(f"Erreur: Paramètre cible '{target_param}' introuvable dans Y.")
    print("Paramètres disponibles:", list(Y.columns))
    exit()

# Vérifier qu'il y a plus d'observations que de prédicteurs
if X_scaled_const.shape[0] <= X_scaled_const.shape[1]:
    print(f"Pas assez d'observations ({X_scaled_const.shape[0]}) pour le nombre de prédicteurs ({X_scaled_const.shape[1]}). Régression impossible.")
    exit()

y_target = Y[target_param]

# Séparer train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled_const, y_target, test_size=0.2, random_state=42, shuffle=False) # Ne pas mélanger pour les séries temporelles

print(f"\nEntraînement du modèle de régression pour {target_param}...")

# Utilisation de statsmodels
try:
    model = sm.OLS(y_train, X_train)
    results = model.fit()
    print(f"\n--- Résultats de la Régression OLS pour {target_param} ---")
    print(results.summary())
    y_train_pred = results.predict(X_train)
    train_rmse = np.sqrt(np.mean((y_train - y_train_pred)**2))

    # Évaluation sur le set de test
    # Assurer que X_test a les mêmes colonnes que X_train
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
    y_pred = results.predict(X_test)
    test_rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    print(f"\nRMSE sur l'ensemble d'entraînement pour {target_param}: {train_rmse:.4f}")
    print(f"\nRMSE sur l'ensemble de test pour {target_param}: {test_rmse:.4f}")
    print(f"Moyenne de {target_param} (test): {y_test.mean():.4f}")
    print(f"Écart-type de {target_param} (test): {y_test.std():.4f}")

    # Plot des résidus vs prédits (simple vérification)
    plt.figure(figsize=(8, 5))
    plt.scatter(results.predict(X_train), results.resid)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Valeurs Prédites (Train)")
    plt.ylabel("Résidus (Train)")
    plt.title(f"Résidus vs Prédits pour {target_param} (Train set)")
    plt.tight_layout()
    plt.show()

    print((y_test- y_pred)**2)
    print((y_train_pred - y_train)**2)

except Exception as e_reg:
     print(f"Erreur lors de l'entraînement/évaluation de la régression: {e_reg}")


print("\n--- Fin du script ---")